# Clasificador español:

Intento de clasificador de convocatorias. Se buscaba evaluar si se aprobaba o no la convocatoria.
A continuación se presenta todo el código fuente del programa:

In [3]:
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

#se requiere instalado el paquete PyStemmer de Snowball
#para hallar las raices de las pal. en español:
import Stemmer
_stemmer = Stemmer.Stemmer('spanish')    #variable global para la funcion tokenize

from sklearn.model_selection import train_test_split


def quitar_puntuacion(text):
 #   return re.sub(ur"[^ÁÉÍÓÚÑÜáéíóúñü\w\d'\s]+",'',text)#,re.UNICODE)
    return re.sub(ur"[^\xc1\xc9\xcd\xd1\xbf\xda\xdc\xe1\xe9\xed\xf1\xf3\xfa\xfc\w\d'\s]+",'',text)

def simplif_tab(text):
    return re.sub(ur"[\t]+"," ",text)

def stem_tokens(tokens, stemmer):
    stemmed = _stemmer.stemWords(tokens)
    return stemmed

def tokenize(text):
    tokens = text.split()
    stems = stem_tokens(tokens, _stemmer)
    return stems


def preprocesamiento(X_train):
    list_convs = [convo.decode('utf-8') for convo in X_train]
    token_dict = {} 

    for (i,conv) in enumerate(list_convs):
        conv = conv.lower()
        conv = simplif_tab(conv) 
        conv_sin_punt = quitar_puntuacion(conv)
        
        token_dict[i] = conv_sin_punt
    return token_dict

def accuracy(y_pred, y_test):
    correct = 0
    total = len(y_test)
    for i, c in enumerate(y_test):
        if(y_pred[i] == c):
            correct += 1
    return correct/float(total)*100

def decod_veredicto(ch_veredicto):
    if(ch_veredicto == 'D'):
        s = "Desaprobado"
    elif(ch_veredicto == 'A'):
        s = "Aprobado"
    else:
        s = ""
    return s

def reporteAcc(token_dict, y_pred, y_test, tope = 10):
    print "Accuracy del clasificador Bayesiano: %.2f%%" %accuracy(y_pred, y_test)
    print
    print "\tCONVOCATORIA\t\t\t\t\t\tPREDICCION\t\tVEREDICTO REAL"
    for i, t in enumerate(y_test.index):
        if(i == tope):
            break
        print "\t", token_dict[i][:30]+"...", "\t\t\t", decod_veredicto(y_pred[i]), \
              "\t\t", decod_veredicto(y_test[t])
def reporte2(token_dict, y_pred, tope = 10):
    print "\tCONVOCATORIA\t\t\t\t\t\tPREDICCION"
    for i, t in enumerate(y_test.index):
        if(i == tope):
            break
        print "\t", token_dict[i][:30]+"...", "\t\t\t", decod_veredicto(y_pred[i])





#Inicio del programa:


#leemos el archivo de entrenamiento (y prueba)
convocs = pd.read_csv('ConvocatoriasRevisadas.csv', sep=',', index_col=False).fillna("")


#seleccionamos los datos objetivo:
convocs["Description"] += " " + convocs["Job Title"] + " " + convocs["Position Level"] \
                              + " " + convocs["Qualifications"]
yy = convocs["Aprobado"]
XX = convocs["Description"]


#separamos datos de entrenamiento


test_prop = 0.2
X_train, X_test, y_train, y_test = train_test_split(XX, yy, test_size = test_prop,    \
                                                    random_state = 23)
###Entrenamiento:
#preprocesamiento:
token_dict = preprocesamiento(X_train)

#transformacion: (usando el tfidf de sklearn con el clasif. Multinomial Naive Bayes)
tfidf = TfidfVectorizer(tokenizer = tokenize)
tfs = tfidf.fit_transform(token_dict.values())

classifier = MultinomialNB(alpha=1.0)
classifier.fit(tfs,y_train)

###Evaluacion del clasificador con la data de prueba (usando accuracy):
token_test = preprocesamiento(X_test)
tfs_pred = tfidf.transform(X_test)
y_pred = classifier.predict(tfs_pred)
reporteAcc(token_test, y_pred, y_test, tope=20)

#hasta aquí todo bien


Accuracy del clasificador Bayesiano: 62.28%

	CONVOCATORIA						PREDICCION		VEREDICTO REAL
	 desarrollo y supervisión de p... 			Desaprobado 		Desaprobado
	elaboración de órdenes de comp... 			Desaprobado 		Desaprobado
	1 desarrollar las soluciones w... 			Desaprobado 		Desaprobado
	1 controlar y supervisar las o... 			Desaprobado 		Aprobado
	1 impulsar la venta de los pro... 			Desaprobado 		Desaprobado
	ingreso de órdenes de compra a... 			Desaprobado 		Aprobado
	1 proporcionar apoyo al área d... 			Desaprobado 		Desaprobado
	 entrenarse a través del proce... 			Aprobado 		Aprobado
	controlar la ejecución de las ... 			Aprobado 		Desaprobado
	 realizar la programación del ... 			Desaprobado 		Aprobado
	1 apoyo en el diseño de subest... 			Desaprobado 		Desaprobado
	 cruzar información en tablas ... 			Desaprobado 		Aprobado
	supervisar y programar las act... 			Desaprobado 		Aprobado
	medir y maximizar  la penetrac... 			Desaprobado 		Desaprobado
	 responsable de la gestión del... 		

Al parecer va todo bien, el clasificador bayesiano tiene un puntaje dentro de lo que se esperaba.
Veamos como se comporta con otros datos de prueba...

In [4]:
#leemos el archivo a clasificar:
new_convocs = pd.read_csv('ConvocatoriasPorRevisar.csv', sep=',', index_col=False).fillna("")
new_convocs["Description"] += " " + new_convocs["Job Title"] + " " + new_convocs["Majors/Concentrations"] \
                              + " " + new_convocs["Qualifications"]

new_X = convocs["Description"]
new_token = preprocesamiento(new_X)
new_tfs = tfidf.transform(X_test)
new_pred = classifier.predict(tfs_pred)
reporte2(new_token, new_pred, len(new_pred))

	CONVOCATORIA						PREDICCION
	 elaboración de ofertas de equ... 			Desaprobado
	ejecutar y liderar las operaci... 			Desaprobado
	por encargo de nuestro cliente... 			Desaprobado
	participar en las distintas et... 			Desaprobado
	planificar y dirigir las activ... 			Desaprobado
	 soporte a las operaciones del... 			Desaprobado
	analizar la demanda de los íte... 			Desaprobado
	nuestro cliente empresa del ru... 			Aprobado
	planificar las actividades de ... 			Aprobado
	gestionar de principio a fin l... 			Desaprobado
	asistir y soportar las activid... 			Desaprobado
	 dominio en htm5  css3 interme... 			Desaprobado
	apoyar en recoger y sistematiz... 			Desaprobado
	generar la oferta del producto... 			Desaprobado
	por encargo de nuestro cliente... 			Desaprobado
	el área de inteligencia de dat... 			Aprobado
	 cumplir con la ejecución del ... 			Desaprobado
	diseño e implementación de cir... 			Aprobado
	          investigar nuevas me... 			Aprobado
	planificar programar y ejecuta... 

Al parecer hay algo extraño con las predicciones... la gran cantidad de desaprobados podría decir que ocurre algo sospechoso...

In [5]:
from collections import Counter
Counter(new_pred)

Counter({'A': 32, 'D': 82})